## Linking without deduplication

A simple record linkage model using the `link_only` [link type](https://moj-analytical-services.github.io/splink/settings_dict_guide.html#link_type).

With `link_only`, only between-dataset record comparisons are generated. No within-dataset record comparisons are created, meaning that the model does not attempt to find within-dataset duplicates.


<a target="_blank" href="https://colab.research.google.com/github/moj-analytical-services/splink/blob/splink4_examples_notebooks/docs/demos/examples/duckdb/link_only.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install git+https://github.com/moj-analytical-services/splink.git@splink4_examples_notebooks

In [2]:
from splink.datasets import splink_datasets

df = splink_datasets.fake_1000

# Split a simple dataset into two, separate datasets which can be linked together.
df_l = df.sample(frac=0.5)
df_r = df.drop(df_l.index)

df_l.head(2)

,unique_id,first_name,surname,dob,city,email,cluster
417,417,Florence,Brown,2002-02-24,London,fb@reose.cem,108
999,999,Oliver,Bird,2000-01-27,Reading,o.b@smith.net,250


In [3]:
from splink.linker import Linker
from splink.duckdb.database_api import DuckDBAPI
from splink.blocking_rule_library import block_on
import splink.comparison_library as cl
import splink.comparison_template_library as ctl
from splink.settings_creator import SettingsCreator

settings = SettingsCreator(
    link_type="link_only",
    blocking_rules_to_generate_predictions=[
        block_on("first_name"),
        block_on("surname"),
    ],
    comparisons=[
        ctl.NameComparison(
            "first_name",
        ),
        ctl.NameComparison("surname"),
        ctl.DateComparison(
            "dob",
            input_is_string=True,
            invalid_dates_as_null=True,
            datetime_metrics=["month", "year", "year"],
            datetime_thresholds=[1, 1, 10],
        ),
        cl.ExactMatch("city").configure(term_frequency_adjustments=True),
        ctl.EmailComparison("email", include_username_fuzzy_level=False),
    ],
)

linker = Linker(
    [df_l, df_r],
    settings,
    database_api=DuckDBAPI(),
    input_table_aliases=["df_left", "df_right"],
)

In [4]:
linker.completeness_chart(cols=["first_name", "surname", "dob", "city", "email"])

alt.LayerChart(...)

In [5]:
from splink.blocking_rule_library import block_on, BlockingRuleCreator

deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    block_on("email"),
]

# TODO: estimate_probability_two_random_records_match should accept
# BlockingRuleCreator objects
deterministic_rules = [
    br.get_blocking_rule("duckdb") if isinstance(br, BlockingRuleCreator) else br
    for br in deterministic_rules
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)

Probability two random records match is estimated to be  0.00326.
This means that amongst all possible pairwise record comparisons, one in 306.48 are expected to match.  With 250,000 total possible comparisons, we expect a total of around 815.71 matching pairs


In [6]:
linker.estimate_u_using_random_sampling(max_pairs=1e6, seed=1)

----- Estimating u probabilities using random sampling -----



Estimated u probabilities using random sampling



Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).


In [7]:
session_dob = linker.estimate_parameters_using_expectation_maximisation(block_on("dob"))
session_email = linker.estimate_parameters_using_expectation_maximisation(
    block_on("email")
)
session_first_name = linker.estimate_parameters_using_expectation_maximisation(
    block_on("first_name")
)


----- Starting EM training session -----



Estimating the m probabilities of the model by blocking on:
l."dob" = r."dob"

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - dob


Iteration 1: Largest change in params was -0.423 in the m_probability of surname, level `Exact match on surname`


Iteration 2: Largest change in params was 0.116 in probability_two_random_records_match


Iteration 3: Largest change in params was 0.0328 in probability_two_random_records_match


...

Iteration 3: Largest change in params was -0.00135 in the m_probability of surname, level `All other comparisons`


Iteration 4: Largest change in params was -0.000242 in the m_probability of surname, level `All other comparisons`


Iteration 5: Largest change in params was 5.77e-05 in the m_probability of email, level `All other comparisons`



EM converged after 5 iterations



Your model is fully trained. All comparisons have at least one estimate for their m and u values


In [8]:
results = linker.predict(threshold_match_probability=0.9)

In [9]:
results.as_pandas_dataframe(limit=5)

,match_weight,match_probability,source_dataset_l,source_dataset_r,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,surname_l,...,dob_l,dob_r,gamma_dob,city_l,city_r,gamma_city,email_l,email_r,gamma_email,match_key
0,3.177090,0.900446,df_left,df_right,525,526,George,George,3,None,...,1980-06-29,1980-06-30,3,None,Lndon,-1,None,go94@brndshaw-saaders.com,-1,0
1,3.184537,0.900908,df_left,df_right,998,997,Oliver,Oliver,3,Bird,...,2000-02-27,2000-01-27,4,None,Reading,-1,oliver.b@smith.net,o.b@smith.net,0,0
2,3.195257,0.901569,df_left,df_right,532,531,ssouiL,Louis,0,Adams,...,1997-09-08,1987-09-11,1,London,None,-1,louis.a@mtrtinez.neea,louis.a@martinez.net,2,1
3,3.374982,0.912085,df_left,df_right,325,324,Kai,Kai,3,Turner,...,2008-12-03,2018-12-31,0,London,London,1,k.t50@hernandez.com,k.t50eherand@z.ncom,1,0
4,3.829434,0.934279,df_left,df_right,941,11,Isabelle,Isla,1,Hall,...,1984-11-20,1985-02-07,2,Swansea,None,-1,None,islahall97@gutierrez-bennett.com,-1,1
